# Multinomial Logit — **Same preprocessing as OLS**, modelling step swapped
This notebook mirrors the OLS pipeline (winsorisation, standardisation, dummies, interaction),
then fits a multinomial logit on `SGrowth_2`. It auto-detects the base category and derives
`High Growth vs Stressed` if needed. Exports odds ratios and marginal effects to Excel.

In [51]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats.mstats import winsorize
from IPython.display import display
import warnings

In [52]:
# === LOAD DATA ===
FILE = 'Results/GRIP Analysis FILTER.xlsx'
df = pd.read_excel(FILE)
print('Loaded:', df.shape)

Loaded: (9050, 81)


In [53]:
# === PARAMETERS (match OLS) ===
target_cont = 'LSGrowth_2023_2019'                 # continuous growth used in OLS
winsor_limits = (0.025, 0.025)                     # 2.5% tails as in OLS
core_cont = ['Starting_INT','Starting_BS_Strength','Starting_Size_ln','Starting_Profit']
cat_target = 'SGrowth_2'                           # categorical for MNLogit
sector_ref = 'budownictwo'                         # OLS reference sector
pf_col = 'P/F'                                     # ownership P/F
hgx_col = 'HGX'                                    # Increased/Decreased Export Intensity


In [54]:
# === WINSORISE CONTINUOUS DV (as in OLS) ===
if target_cont in df.columns:
    arr = df[target_cont].astype(float).to_numpy()
    df[target_cont + '_win'] = winsorize(arr, limits=winsor_limits)
    print('Winsorised DV created:', target_cont + '_win')
else:
    print('Warning: continuous OLS DV not found; continuing with multinomial anyway.')

Winsorised DV created: LSGrowth_2023_2019_win


In [55]:
# === STANDARDISE core continuous predictors (z-score), as in OLS ===
for c in core_cont:
    if c in df.columns:
        mu, sd = df[c].mean(), df[c].std(ddof=0)
        df[c + '_z'] = (df[c] - mu) / (sd if sd else 1.0)
    else:
        raise KeyError(f'Missing predictor: {c}')
print('Standardised predictors:', [c + '_z' for c in core_cont])


Standardised predictors: ['Starting_INT_z', 'Starting_BS_Strength_z', 'Starting_Size_ln_z', 'Starting_Profit_z']


In [56]:
# === DUMMIES to match OLS ===
# Foreign dummy from P/F (F=1, P=0)
df['Foreign'] = (df[pf_col].astype(str).str.upper().str.strip() == 'F').astype(int)

# HGX Increased Export Intensity dummy (reference = Decreased)
df['HGX_Increased Export Intensity'] = (df[hgx_col].astype(str).str.strip() == 'Increased Export Intensity').astype(int)

# Sector dummies, drop reference sector
sector_d = pd.get_dummies(df['Sector'], prefix='Sector', drop_first=False)
ref_col = f'Sector_{sector_ref}'
if ref_col in sector_d.columns:
    sector_d = sector_d.drop(columns=[ref_col])
print('Sector dummies:', len(sector_d.columns), ' (ref =', sector_ref, ')')


Sector dummies: 13  (ref = budownictwo )


In [57]:
# === INTERACTION (as in OLS) ===
df['Starting_INT_Mult_Starting_Size_ln'] = df['Starting_INT'] * df['Starting_Size_ln']

# Collect the design matrix columns mirroring OLS structure (z-scored cores + dummies + interaction)
X_cols = [c + '_z' for c in core_cont] + ['Foreign','HGX_Increased Export Intensity','Starting_INT_Mult_Starting_Size_ln']
X = pd.concat([df[X_cols], sector_d], axis=1)

# Align with categorical target
cat_map = {'0.Stressed':0,'1.Declining':1,'2.Stable':2,'3.High Growth':3}
y = df[cat_target].map(cat_map)
mask = (~y.isna()) & (~X.isna().any(axis=1))
X = X.loc[mask].copy()
y = y.loc[mask].astype(int)
print('Model N:', X.shape[0], 'Features:', X.shape[1])


Model N: 9015 Features: 20


In [58]:
# === SAFETY: ensure numeric dtypes to avoid object errors ===
Xc = sm.add_constant(X, has_constant='add')
Xc = Xc.astype(float)
y = y.astype(int)
print('dtypes:', Xc.dtypes.value_counts().to_dict())


dtypes: {dtype('float64'): 21}


In [74]:
# === Minimal Multinomial Logit: clean, quiet, results only ===
# Inputs: X (DataFrame of predictors), y (Series/array of integer labels)

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# 1) Prepare design matrix
Xc = sm.add_constant(X, has_constant='add').astype(float)
y  = pd.Series(y).astype(int)

# Drop zero-variance predictors except intercept
dropcols = [c for c in Xc.columns if c != 'const' and Xc[c].nunique(dropna=True) <= 1]
if dropcols:
    Xc = Xc.drop(columns=dropcols)

# Standardize non-intercept columns
cols = [c for c in Xc.columns if c != 'const']
Xc[cols] = (Xc[cols] - Xc[cols].mean()) / Xc[cols].std(ddof=0)

mnl = sm.MNLogit(y, Xc)

#2) Try unregularized fit first
try:
    res = mnl.fit(method="lbfgs", maxiter=2000, disp=False, cov_type="HC1")
except (TypeError, np.linalg.LinAlgError):
    res = mnl.fit(method="lbfgs", maxiter=2000, disp=False)

# 3) If not converged, apply gentle ridge and refit for SEs
if not bool(res.mle_retvals.get("converged", True)):
    res_reg = mnl.fit_regularized(method="lbfgs", alpha=0.5, L1_wt=0.0, maxiter=4000, disp=False)
    start = np.asarray(res_reg.params).ravel()
    try:
        res = mnl.fit(method="lbfgs", start_params=start, maxiter=4000, disp=False, cov_type="HC1")
    except TypeError:
        res = mnl.fit(method="lbfgs", start_params=start, maxiter=4000, disp=False)

# 4) Display results only
try:
    display(res.summary())      # or display(res.summary2()) for a wider table
except Exception:
    print(res.summary())
print(res.params.index)

<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:              SGrowth_2   No. Observations:                 9015
Model:                        MNLogit   Df Residuals:                     8952
Method:                           MLE   Df Model:                           60
Date:                Thu, 30 Oct 2025   Pseudo R-squ.:                 0.04743
Time:                        20:49:44   Log-Likelihood:                -10655.
converged:                       True   LL-Null:                       -11185.
Covariance Type:                  HC1   LLR p-value:                4.960e-183
======================================================================================================
                       SGrowth_2=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                  1.1450      0.039     29.371      0.000       1.069       1.221
Starting_INT_z                         0.2993      0.569      0.526      0.599      -0.816       1.414
Starting_BS_Strength_z                 0.2932      0.045      6.453      0.000       0.204       0.382
Starting_Size_ln_z                     0.1470      0.049      2.969      0.003       0.050       0.244
Starting_Profit_z                      0.1077      0.052      2.074      0.038       0.006       0.209
Foreign                               -0.0220      0.037     -0.601      0.548      -0.094       0.050
HGX_Increased Export Intensity         0.0190      0.035      0.541      0.588      -0.050       0.088
Starting_INT_Mult_Starting_Size_ln    -0.3499      0.572     -0.611      0.541      -1.472       0.772
Sector_chemia                          0.0845      0.038      2.244      0.025       0.011       0.158
Sector_energetyka                     -0.0954      0.046     -2.079      0.038      -0.185      -0.005
Sector_górnictwo i hutnictwo           0.1084      0.037      2.957      0.003       0.037       0.180
Sector_handel detaliczny               0.0821      0.038      2.157      0.031       0.007       0.157
Sector_handel hurtowy                  0.0166      0.040      0.416      0.677      -0.062       0.095
Sector_media, telekomunkacja, IT       0.2625      0.042      6.269      0.000       0.180       0.345
Sector_motoryzacja                     0.2468      0.044      5.605      0.000       0.160       0.333
Sector_ochrona zdrowia i farmacja      0.1942      0.041      4.693      0.000       0.113       0.275
Sector_paliwa                         -0.0467      0.034     -1.373      0.170      -0.113       0.020
Sector_produkcja                       0.2380      0.049      4.904      0.000       0.143       0.333
Sector_transport                       0.1858      0.042      4.453      0.000       0.104       0.268
Sector_usługi                          0.2109      0.040      5.279      0.000       0.133       0.289
Sector_żywność                         0.5648      0.066      8.613      0.000       0.436       0.693
------------------------------------------------------------------------------------------------------
                       SGrowth_2=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                  1.1909      0.039     30.639      0.000       1.115       1.267
Starting_INT_z                         1.8425      0.563      3.270      0.001       0.738       2.947
Starting_BS_Strength_z                 0.3288      0.043      7.635      0.000       0.244       0.413
Starting_Size_ln_z                     0.3066      0.047      6.520      0.000       0.214       0.399
Starting_Profit_z                      0.1986   

Index(['const', 'Starting_INT_z', 'Starting_BS_Strength_z',
       'Starting_Size_ln_z', 'Starting_Profit_z', 'Foreign',
       'HGX_Increased Export Intensity', 'Starting_INT_Mult_Starting_Size_ln',
       'Sector_chemia', 'Sector_energetyka', 'Sector_górnictwo i hutnictwo',
       'Sector_handel detaliczny', 'Sector_handel hurtowy',
       'Sector_media, telekomunkacja, IT', 'Sector_motoryzacja',
       'Sector_ochrona zdrowia i farmacja', 'Sector_paliwa',
       'Sector_produkcja', 'Sector_transport', 'Sector_usługi',
       'Sector_żywność'],
      dtype='object')


In [71]:
# === MARGINAL EFFECTS (average dydx) ===

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

mfx = res.get_margeff(at='overall', method='dydx').summary_frame()

# display the whole thing
display(mfx)

mfx.to_excel("Results/Regression MNL Marginal Effects.xlsx", index=True)

dy/dx  Std. Err.  \
endog       exog                                                      
SGrowth_2=0 Starting_INT_z                     -0.157405   0.058554   
            Starting_BS_Strength_z             -0.031356   0.004531   
            Starting_Size_ln_z                 -0.026328   0.004983   
            Starting_Profit_z                  -0.017541   0.005647   
            Foreign                             0.010042   0.003797   
            HGX_Increased Export Intensity     -0.009840   0.003639   
            Starting_INT_Mult_Starting_Size_ln  0.168766   0.058949   
            Sector_chemia                      -0.007354   0.003886   
            Sector_energetyka                   0.008269   0.004256   
            Sector_górnictwo i hutnictwo        0.010049   0.003873   
            Sector_handel detaliczny           -0.005440   0.003899   
            Sector_handel hurtowy              -0.002130   0.003919   
            Sector_media, telekomunkacja, IT   -0.020732   0.004396   
            Sector_motoryzacja                 -0.022511   0.004467   
            Sector_ochrona zdrowia i farmacja  -0.019540   0.004296   
            Sector_paliwa                       0.005391   0.003272   
            Sector_produkcja                   -0.017690   0.004911   
            Sector_transport                   -0.019738   0.004281   
            Sector_usługi                      -0.018277   0.004114   
            Sector_żywność                     -0.068736   0.007061   
SGrowth_2=1 Starting_INT_z                     -0.325030   0.087522   
            Starting_BS_Strength_z              0.017283   0.006308   
            Starting_Size_ln_z                 -0.020688   0.007360   
            Starting_Profit_z                  -0.010258   0.006181   
            Foreign                             0.019518   0.005336   
            HGX_Increased Export Intensity     -0.020730   0.005062   
            Starting_INT_Mult_Starting_Size_ln  0.339322   0.087801   
            Sector_chemia                       0.009083   0.005807   
            Sector_energetyka                  -0.009250   0.008886   
            Sector_górnictwo i hutnictwo        0.060832   0.005131   
            Sector_handel detaliczny            0.015294   0.005860   
            Sector_handel hurtowy               0.000410   0.006836   
            Sector_media, telekomunkacja, IT    0.038317   0.005877   
            Sector_motoryzacja                  0.027203   0.007049   
            Sector_ochrona zdrowia i farmacja   0.015995   0.005808   
            Sector_paliwa                      -0.000977   0.006075   
            Sector_produkcja                    0.037147   0.007817   
            Sector_transport                    0.011683   0.006075   
            Sector_usługi                       0.026386   0.005884   
            Sector_żywność                      0.012329   0.007219   
SGrowth_2=2 Starting_INT_z                      0.238755   0.086494   
            Starting_BS_Strength_z              0.030790   0.005756   
            Starting_Size_ln_z                  0.038057   0.006809   
            Starting_Profit_z                   0.023214   0.006351   
            Foreign                            -0.012082   0.005440   
            HGX_Increased Export Intensity      0.022673   0.005058   
            Starting_INT_Mult_Starting_Size_ln -0.268636   0.086735   
            Sector_chemia                       0.013338   0.005589   
            Sector_energetyka                  -0.031162   0.009055   
            Sector_górnictwo i hutnictwo        0.044670   0.005142   
            Sector_handel detaliczny           -0.016534   0.005984   
            Sector_handel hurtowy              -0.003491   0.006541   
            Sector_media, telekomunkacja, IT   -0.025733   0.006228   
            Sector_motoryzacja                 -0.010728   0.007106   
            Sector_ochrona zdrowia i farmacja   0.000740   0.005726   
    

In [72]:
# === Export MNLogit coefficients with correct outcome labels (matches summary) ===
import pandas as pd
import numpy as np

yname = getattr(y, "name", "y") or "y"

# 1) Collect the matrices
coef  = res.params
se    = res.bse
zval  = res.tvalues
pval  = res.pvalues
ci_lo = coef - 1.96 * se
ci_hi = coef + 1.96 * se

# 2) Determine the true outcome labels
# Statsmodels MNLogit drops one base category (usually the last in sorted order)
# The remaining fitted equations correspond to the other categories in sorted order
cats = pd.Series(y).dropna().unique()
cats = np.sort(cats)                       # ensure sorted numeric order
base = cats[-1]                            # last one = base category
nonbase = [c for c in cats if c != base]

# Now map each column of coef (in order) to its actual category label
if len(nonbase) != len(coef.columns):
    # fallback if categories somehow mismatched
    outcome_labels = [f"{yname}={c}" for c in coef.columns]
else:
    outcome_labels = [f"{yname}={c}" for c in nonbase]

outcome_map = {col: label for col, label in zip(coef.columns, outcome_labels)}

# 3) Stack for tidy alignment
def S(df, name):
    s = df.stack()
    s.name = name
    return s

stacked = pd.concat([
    S(coef,  "coef"),
    S(se,    "std err"),
    S(zval,  "z"),
    S(pval,  "P>|z|"),
    S(ci_lo, "[0.025"),
    S(ci_hi, "0.975]"),
], axis=1).reset_index().rename(columns={"level_0":"Variable", "level_1":"Outcome"})

# 4) Apply the proper outcome labels
stacked["Outcome"] = stacked["Outcome"].map(outcome_map)

# 5) Sort and export
stacked["Outcome"] = pd.Categorical(stacked["Outcome"], categories=outcome_labels, ordered=True)
stacked["Variable"] = pd.Categorical(stacked["Variable"], categories=list(coef.index), ordered=True)
coef_tidy = stacked.sort_values(["Outcome", "Variable"]).reset_index(drop=True)

out_path = "mnlogit_coefficients_tidy.xlsx"
with pd.ExcelWriter(out_path, engine="xlsxwriter") as xw:
    coef_tidy.to_excel(xw, sheet_name="Coefficients", index=False)

print(f"Exported → {out_path}")

Exported → mnlogit_coefficients_tidy.xlsx


Notes:
- Preprocessing matches OLS: **winsorisation**, **z-scored predictors**, **Foreign**, **HGX Increased**, **sector** dummies, **INT×Size**.
- Model step is swapped to **MNLogit**.
- If the base is High Growth (3), the notebook also **derives High Growth vs Stressed** ORs automatically.
- All predictors are coerced to float before fitting to avoid dtype errors.